In [1]:
import sys
import os
import pandas as pd
import numpy as np
from datetime import datetime
import pymysql



from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
HOSTCaris =os.getenv("HOSTCaris")
DBCaris =os.getenv("DBCaris")
USRCaris =os.getenv("USRCaris")
PASSCaris =os.getenv("PASSCaris")

connection = pymysql.connect(
    host=HOSTCaris,
    user=USRCaris,
    password=PASSCaris,
    db=DBCaris
)

In [4]:
req='''SELECT *
FROM
    caris_db.view_muso_household_report
where (isnull(indice_householdcount) and (is_household_applicable ='no' or isnull(is_household_applicable)))'''

look_up_caseid="""
SELECT 
    mgm.id_patient,
    p.patient_code as patient_code_onTable,
    p.muso_case_id as caseid
FROM
    caris_db.muso_group_members mgm
        LEFT JOIN
    caris_db.patient p ON mgm.id_patient = p.id;
"""

In [5]:
compt=pd.read_sql(req,connection)
look_up_caseid= pd.read_sql(look_up_caseid,connection)



connection.close()

/usr/local/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/usr/local/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [6]:
pvvih_muso_cm_ptme = pd.read_excel('../pvvih_sansComptage/rapport_pvvih_cm_ptme.xlsx')

In [7]:
compt.columns


Index(['indice_householdcount', 'id_patient', 'prenom', 'nom', 'phone',
       'patient_code', 'id_group', 'group_name', 'office', 'commune', 'code',
       'case_id_groupes', 'groups_closed_on_commcare',
       'is_household_applicable', 'is_inactive_groupes',
       'is_graduated_groupes', 'is_inactive_members', 'is_graduated_members',
       'members_closed_on_commcare'],
      dtype='object')

In [8]:
pvvih_muso_cm_ptme.rename(
    columns={
        'OFFICE': 'office'
    },
    inplace=True
)
pvvih_muso_cm_ptme.columns

Index(['Menage PTME', 'caseid', 'patient_code_pv', 'code_labo', 'is_pvvih',
       'group_name', 'prenom et nom', 'prenom', 'nom', 'dob', 'office',
       'id_groupes'],
      dtype='object')

In [9]:
look_up_caseid.columns

Index(['id_patient', 'patient_code_onTable', 'caseid'], dtype='object')

In [10]:
compt =  compt.merge(look_up_caseid[['id_patient','caseid']],on="id_patient",how="left")
compt.columns

Index(['indice_householdcount', 'id_patient', 'prenom', 'nom', 'phone',
       'patient_code', 'id_group', 'group_name', 'office', 'commune', 'code',
       'case_id_groupes', 'groups_closed_on_commcare',
       'is_household_applicable', 'is_inactive_groupes',
       'is_graduated_groupes', 'is_inactive_members', 'is_graduated_members',
       'members_closed_on_commcare', 'caseid'],
      dtype='object')

In [11]:
compt = compt[compt.group_name != 'test Shisu']

In [12]:
compt.caseid.count()
compt.id_patient.count()
compt.pivot_table(
    values="id_patient",
    index="office",
    aggfunc='count',
    fill_value=0,
    margins=True,
    margins_name="Total"
)

990

990

,id_patient
office,
CAP,597
CAY,19
GON,310
JER,27
PAP,37
Total,990


In [13]:
# remove pcmp in compt
remove_compt_pmcmp = compt[compt.caseid.isin(pvvih_muso_cm_ptme.caseid)]
compt = compt[~compt.caseid.isin(pvvih_muso_cm_ptme.caseid)]

remove_compt_pmcmp.id_patient.count()
compt.id_patient.count()

129

861

In [14]:
resume = compt.pivot_table(
    values="id_patient",
    index="office",
    aggfunc='count',
    fill_value=0,
    margins=True,
    margins_name="Total"
)
resume


resume_remove = remove_compt_pmcmp.pivot_table(
    values="id_patient",
    index="office",
    aggfunc='count',
    fill_value=0,
    margins=True,
    margins_name="Total"
)
resume_remove

,id_patient
office,
CAP,534
CAY,6
GON,294
JER,27
Total,861


,id_patient
office,
CAP,63
CAY,13
GON,16
PAP,37
Total,129


In [15]:
resume_pmcp = pvvih_muso_cm_ptme.pivot_table(
    values="caseid",
    index="office",
    aggfunc='count',
    fill_value=0,
    margins=True,
    margins_name="Total"
)

resume_pmcp

,caseid
office,
CAP,110
GON,28
PAP,104
PDP,5
Total,247


In [16]:
LOCATION_ALL_PVVIH_MUSO_CM_ptme=[
    f"../pvvih_sansComptage/total pvvih muso avec comptage ptme.xlsx",
    f"./total pvvih muso avec comptage ptme.xlsx"
]

LOCATION_IDENTIFIED_PVVIH_MUSO_CM_ptme=[
    f"../pvvih_sansComptage/pvvih muso avec comptage ptme identifie dans comptage muso restant.xlsx",
    f"./total pvvih muso avec comptage ptme identifie dans comptage muso restant.xlsx"
]

In [17]:
writer = pd.ExcelWriter(f"Comptage non faits_{str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", engine='xlsxwriter')
resume.to_excel(writer,index=True,sheet_name="resume")
for name in compt["office"].unique().tolist():
    file=compt[compt["office"]== name]
    file.to_excel(writer, index=False, sheet_name=f'{name}')
writer.save()

In [18]:
for location in LOCATION_ALL_PVVIH_MUSO_CM_ptme:
    writer = pd.ExcelWriter(location, engine='xlsxwriter')
    resume_pmcp.to_excel(writer,index=True,sheet_name="resume")
    for name in pvvih_muso_cm_ptme["office"].unique().tolist():
        file=pvvih_muso_cm_ptme[pvvih_muso_cm_ptme["office"]== name]
        file.to_excel(writer, index=False, sheet_name=f'{name}')
    writer.save()

In [19]:
for loc in LOCATION_IDENTIFIED_PVVIH_MUSO_CM_ptme:
    writer = pd.ExcelWriter(loc, engine='xlsxwriter')
    resume_remove.to_excel(writer,index=True,sheet_name="resume")
    for name in remove_compt_pmcmp["office"].unique().tolist():
        file=remove_compt_pmcmp[remove_compt_pmcmp["office"]== name]
        file.to_excel(writer, index=False, sheet_name=f'{name}')
    writer.save()

In [ ]:
compt.to_excel('cnf_muso.xlsx',index=False,na_rep="")